
<font size=20>
Insert filter here:


In [ ]:
assignee = "shell oil company"

<font size=20>
Import Packages


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt  
import matplotlib.animation 
#import matplotlib
#matplotlib.use('TkAgg')
import numpy as np

import requests 

import os
import pandas as pd
import psycopg2
#import contextily as ctx
#from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import chain


<font size=20>read files inside folder for translational data

In [ ]:
#read files inside folder for translational data


from typing import List
import pathlib
wd = os.getcwd()
wd = wd+"/Prepared_Data"
def find_excel_files_in(directory:pathlib.Path) -> List[pathlib.Path]:
    
    files:List[pathlib.Path] = list()
    for filepath in directory.rglob('*.csv'):
        if filepath.is_file():
            files.append(filepath)

    return files

# List of your directories
directories:List[str] = [format(wd)]

found_files:List[pathlib.Path] = list()

for directory in directories:
    directory:pathlib.Path = pathlib.Path(directory)
    found_files.extend(find_excel_files_in(directory))

#read excel files
import pandas as pd

#print(files)
    #Deleting all empty rows in the file df
for files in found_files:
    data = pd.read_csv(files)

#filter for assignee/applicants


#Prepare a compare column for assignee filter
assignee2 = assignee.lower()
assignee2 = assignee2.replace(" ","")
data["compare"] = data["optimized_assignee"]
data["compare"] =data["compare"].str.lower()
data["compare"] =data["compare"].str.replace(" ","")
#converting inpadoc_family_members back to list
data["inpadoc_family_members"] = data["inpadoc_family_members"].str.replace("'","")
data["inpadoc_family_members"] = data["inpadoc_family_members"].str.replace("[","")
data["inpadoc_family_members"] = data["inpadoc_family_members"].str.replace("]","")
data["inpadoc_family_members"] = data["inpadoc_family_members"].str.replace(" ","")
data["inpadoc_family_members"] = data["inpadoc_family_members"].fillna("")
data["inpadoc_family_members"] = data.apply(lambda x: x["inpadoc_family_members"] 
                                                        if len(x["inpadoc_family_members"]) > 4 
                                                        else "",axis=1 )
#check if publication number has been added to inpadoc_family_members and if not add it
data["inpadoc_family_members"] = data.apply(lambda x: str(x["publication_number"])+ " | " +str(x["inpadoc_family_members"]) 
                                            if str(x["publication_number"]) not in str(x["inpadoc_family_members"]) else str(x["inpadoc_family_members"])
                                            ,axis = 1)

#prepare inpadoc_family_members_column
data["inpadoc_family_members"] = data.apply(lambda x: str(x["inpadoc_family_members"]).split(","),axis = 1)

#data["inpadoc_family_members"] = data.apply(lambda x: x["inpadoc_family_members_list"].split(","),axis = 1)

#apply filter
data = data[data["compare"]==assignee2]
data.drop("compare",axis = 1,inplace = True)
#inpadoc_family_members_list = list(pre_data["inpadoc_family_members"])

#look for other members in the family
#inpadoc_family_members_list = rerun_slower(inpadoc_family_members_list,data)
#data[data["publication_number"].isin(inpadoc_family_members_list)]

#creation of comparable dataframe
data_compare = data


#creation of df containg every family member as entry even tho some data might not exists 
publication_number = []
inpadoc_family = []
listdata = list(data.inpadoc_family_members)
for i in range(len(listdata)):
    for k in range(len(listdata[i])):
        #print(str(i)+"   "+str(k)+"list:"+listdata[i][k])
        publication_number.append(listdata[i][k])
        inpadoc_family.append(listdata[i])
d = {'publication_number': publication_number,
    'inpadoc_family_members': inpadoc_family}
data_family = pd.DataFrame(data=d)
#make column inpadoc_family_members mergable
data["inpadoc_family_members"] = data["inpadoc_family_members"].astype("string")
data_family["inpadoc_family_members"] = data_family["inpadoc_family_members"].astype("string")
#merge
data_family = data_family.merge(data_compare, right_on=['publication_number','inpadoc_family_members'],left_on=['publication_number','inpadoc_family_members'],how="left")

data_family.reset_index(inplace= True)
data_family = data_family.drop("index", axis = 1)
#drop duplicates
data_family = data_family.drop_duplicates(subset=["publication_number"], keep="first")
data = data_family

#DATA ADJUSTMENTS

#read files inside folder for translation of publication Number 

wd = os.getcwd()
from typing import List
import pathlib
wd2 = wd + "/Translationaldata"
def find_excel_files_in(directory:pathlib.Path) -> List[pathlib.Path]:
    
    files:List[pathlib.Path] = list()
    for filepath in directory.rglob('*.xlsx'):
        if filepath.is_file():
            files.append(filepath)

    return files

# List of your directories
directories:List[str] = [format(wd2)]

found_files:List[pathlib.Path] = list()

for directory in directories:
    directory:pathlib.Path = pathlib.Path(directory)
    found_files.extend(find_excel_files_in(directory))

#read excel files
import pandas as pd

#print(files)
    #Deleting all empty rows in the file df
for files in found_files:
    xls = pd.ExcelFile(files)

file = pd.read_excel(xls, "PUBLNR",skiprows=2)

filter = file["DOCDB"] != " "
file = file[filter]

#Filling all empty cells in the country code(CC) column inside file df

for x in range(len(file)):
    if file.iloc[x][0] != " ":
        shortcode_for_country = file.iloc[x][0]
    if file.iloc[x][0] == " ":
        file.iloc[x][0] = shortcode_for_country
    if file.iloc[x][1] != " ":
        countryname = file.iloc[x][1]
    if file.iloc[x][1] == " ":
        file.iloc[x][1] = countryname




#read ST.30 Sheet in excel file for creating "simplified_status",patent_status and patent_status_description
df1_additional = pd.read_excel(xls, 'ST30_Pub availability',skiprows = 2)
df1_additional = df1_additional[["Unnamed: 0", "Kindcode", "ST.30"]].astype("string").fillna(" ")
df1_additional.reset_index(inplace= True)
df1_additional = df1_additional.drop("index", axis = 1)
df1_additional["Kindcode"] = df1_additional["Kindcode"].str.replace(" ","")
# #Filling all empty cells in the country code(CC) column

for x in range(len(df1_additional)):
    if df1_additional.iloc[x][0] != " ":
        shortcode_for_country = df1_additional.iloc[x][0]
    if df1_additional.iloc[x][0] == " ":
        df1_additional.loc[df1_additional.index[x], 'Unnamed: 0'] = f"{shortcode_for_country}"

#Concatenating all different types of documents into one Column

file = file.groupby(['CC','Description','DOCDB'])['Type of document'].apply(', '.join).reset_index()

for col in ["Type of document"]:
    file[col]=file[col].str.split(", ").map(set).str.join(", ")
file["Type of document"] = file["Type of document"].str.replace(', ','| ')

#Translation summary of the type of document
df1_translation = pd.read_excel(xls, 'ST30_Pub availability',skiprows = 0)
df1_translation = df1_translation[["ST.30","Description XML element"]].astype("string").fillna(" ")
filter = df1_translation["ST.30"] != " "
df1_translation = df1_translation[filter]

#Comparing columns from both data frames(file and data) to fill the column 'Country' in the data df
data.drop("country",axis = 1,inplace = True)
data["Country"] = ""
for i in range(len(data.publication_number)):
    #print(f'1 {i}')
    for x in range(len(file.CC)):
        #print(f'2 {x}')
        if data["publication_number"].iloc[i][0:2] == file.CC.iloc[x]:
            #print(f'3 {data["publication_number"].iloc[i][0:2]}')
            #print(f'4 {file.CC.iloc[x]}')  
            data.loc[i,'Country'] = file.loc[x,"Description"]
            break


#Inserting a "Type of document" column
#Comparing the last two indexes to the DOCDB column to determine the type of document
#Comparing the last index to the DOCDB column to determine the type of document
data["type_of_document"] = ""

for i in range(len(data.publication_number)):
    #print(f'1 {i}')
    for x in range(len(file.DOCDB)):
        #print(f'2 {x}')
        if data["publication_number"].iloc[i][-2:]  == file.DOCDB.iloc[x] and data["Country"].iloc[i] == file.Description.iloc[x]:
            #print(f'3 {data["publication_number"].iloc[i][0:2]}')
            #print(f'4 {file.CC.iloc[x]}')  
            #data['type_of_document'].iloc[i] = file["Type of document"].iloc[x]
            
            data.loc[i,"type_of_document"] = file.loc[x,"Type of document"]
            break
        if data["publication_number"].iloc[i][-1:]  == file.DOCDB.iloc[x] and data["Country"].iloc[i] == file.Description.iloc[x]:
            #print(f'3 {data["publication_number"].iloc[i][0:2]}')
            #print(f'4 {file.CC.iloc[x]}')  
            data.loc[i,"type_of_document"] = file.loc[x,"Type of document"]

            break
        
#Making a new column comparing two df
#The Kindcode from df1_additional df and publication number from the data df
#Numbers compared and inserted(450,470, A44 etc), each number represents a type
#of document

data["ST.30"] = ""
for i in range(len(data.publication_number)):
    #print(f'1 {i}')
    for x in range(len(df1_additional)):
        #print(f'2 {x}')
        if data["publication_number"].iloc[i][-2:]  == df1_additional["Kindcode"].iloc[x]:
            if data["publication_number"].iloc[i][0:2] == df1_additional["Unnamed: 0"].iloc[x]:
            #print(f'3 {data["publication_number"].iloc[i][0:2]}')
            #print(f'4 {file.CC.iloc[x]}')  
                data.loc[i,'ST.30'] = df1_additional.loc[x,'ST.30']
                break
        if data["publication_number"].iloc[i][-1:]  == df1_additional["Kindcode"].iloc[x]:
            if data["publication_number"].iloc[i][0:2] == df1_additional["Unnamed: 0"].iloc[x]:
            #print(f'3 {data["publication_number"].iloc[i][0:2]}')
            #print(f'4 {file.CC.iloc[x]}')  
                data.loc[i,'ST.30'] = df1_additional.loc[x,'ST.30']
                break

#create simplified_status column; look at df1_translation for changes
data["simplified_status"]= data.apply(lambda x: "granted" if x["ST.30"]== "450" or x["ST.30"]=="470"
                                                         else "not granted",axis=1)
                            
data["patent_status_description"] = ""
for i in range(len(data.publication_number)):

    for x in range(len(df1_translation)):
  
        if data["ST.30"].iloc[i]  == df1_translation["ST.30"].iloc[x]:
            data.loc[i,"patent_status_description"] = df1_translation.loc[x,'Description XML element']
            break
#make data columns lowercase
data.columns = data.columns.str.lower()
#hong kong to china in order to be displayed in worldmap later
data["country"] = data.apply(lambda x: "china" if x["country"]== 'hong kong (s.a.r.)' else x["country"],axis =1)



#create data for total and add it to data df
#Creating 3 dataframes: All, Granted, Not-Granted with new column total_of_publication_number respectively
from pandas import NA

total_column = data[data['country'].values != ""]
total_column = total_column[["publication_number","country"]].groupby('country').agg('count')
total_column.rename(columns = {"publication_number" : "total_of_publication_number"}, inplace = True)
total_column.reset_index(inplace = True)
total_column["total_of_publication_number"] = total_column.apply(lambda x: NA if x["total_of_publication_number"] == 0 else x["total_of_publication_number"],axis = 1)
data_all_total = data.merge(total_column, left_on = 'country', right_on = 'country', how='left')
#data_all_total

#create data for proportion and add it to data df BUT FOR GRANTED ONLY
prestep= data[data['country'].values != ""]
total_column = prestep[prestep["simplified_status"] == "granted"]
if len(total_column) == 0:
    data_granted_total = None
    print("WARNING: no patents are granted with this filter")
else:
    total_column = total_column[["publication_number","country"]].groupby('country').agg('count')
    total_column.rename(columns = {"publication_number" : "total_of_publication_number"}, inplace = True)
    total_column.reset_index(inplace = True)
    total_column["total_of_publication_number"] = total_column.apply(lambda x: NA if x["total_of_publication_number"] == 0 else x["total_of_publication_number"],axis = 1)
    data_granted_total = data.merge(total_column, left_on = 'country', right_on = 'country', how='left')
    #data_granted_total

#create data for proportion and add it to data df BUT FOR NOT-GRANTED ONLY
prestep= data[data['country'].values != ""]
total_column = prestep[prestep["simplified_status"] == "not granted"]
if len(total_column) == 0:
    data_not_granted_total = None
    print("WARNING: no patents are not-granted with this filter")
    
else:
    total_column = total_column[["publication_number","country"]].groupby('country').agg('count')
    total_column.rename(columns = {"publication_number" : "total_of_publication_number"}, inplace = True)
    total_column.reset_index(inplace = True)
    total_column["total_of_publication_number"] = total_column.apply(lambda x: NA if x["total_of_publication_number"] == 0 else x["total_of_publication_number"],axis = 1)
    data_not_granted_total = data.merge(total_column, left_on = 'country', right_on = 'country', how='left')
    #data_not_granted_total



#get current working directory

current_wd = os.getcwd()
path_worldmap= "/Worldmap/World_Countries.shp"
path_worldmap = current_wd+path_worldmap
#read worldmap shape file
world_map = gpd.read_file(path_worldmap)
#make country names lowercase in order to match dataframes by countrynames
world_map["COUNTRY"]=world_map["COUNTRY"].str.lower()
world_map.sort_values(by = "COUNTRY")
#adjust important names for merging
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("united states", "united states of america")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("south korea", "korea (south)")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("moldova", "republic of moldova")
world_map["COUNTRY"] = world_map["COUNTRY"].str.replace("russia", "russian federation")

#merge dataframes
if data_all_total is not None:  
    merged_all = world_map.merge(data_all_total, left_on = 'COUNTRY', right_on = 'country', how='outer')
    merged_all['total_of_publication_number'] = merged_all['total_of_publication_number'].fillna(0)
else:
    print("WARNING: no patents exists using this filter")
#merge granted
if data_granted_total is not None:
    merged_granted = world_map.merge(data_granted_total, left_on = 'COUNTRY', right_on = 'country', how='outer')
    merged_granted['total_of_publication_number'] = merged_granted['total_of_publication_number'].fillna(0)
else:
    print("WARNING: no patents are granted using this filter")
if data_not_granted_total is not None:
    merged_not_granted = world_map.merge(data_not_granted_total, left_on = 'COUNTRY', right_on = 'country', how='outer')
    merged_not_granted['total_of_publication_number'] = merged_not_granted['total_of_publication_number'].fillna(0)
else:
    print("WARNING: no patents are not granted using this filter")


#create list of countries that are not on the world map

countries_not_on_map = []
for element in list(total_column["country"]):
    if element not in list(world_map["COUNTRY"]):
        countries_not_on_map.append(element)
 
print("Values that can't be shown in the plot, since they are not assigned to a country: \n"+str(countries_not_on_map))

#extract proportions for if "country" is no actual country
nocountries = total_column["country"].isin(countries_not_on_map)
nocountries = total_column[nocountries]

<font size=20>Check if filter works

In [ ]:
if len(data) == 0:
    print("WARNING: no data available using this filter")
    print("number of patents in the dataframe: "+str(len(data)))
else:
    print("filter application works")
    print("number of patents in the dataframe: "+str(len(data)))

<font size=20>Creation of Visualizations

In [ ]:
import math
def round_up_to_nearest_10(num):
    return math.ceil(num / 10) * 10

round_up_to_nearest_10(21)

#plot worldmap ALL
def plot_worldmap(df,situational,status):
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib import cm
    from matplotlib.colors import TwoSlopeNorm
    import statistics

    #statistics.median(percentage_column["proportion_of_publication_number"])


    fig, ax = plt.subplots(figsize = (20,10))

    #divider = make_axes_locatable(ax)

    #cax = divider.append_axes("right", size="5%", pad=0.1)

    # create a normalized colorbar
    #norm = TwoSlopeNorm(vmin=min(total_column["total_of_publication_number"]),
    #                vcenter=statistics.median(total_column["total_of_publication_number"]),
    #                vmax=max(total_column["total_of_publication_number"]))
    cmap=plt.cm.get_cmap('plasma', 50)
    #cbar = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

    #creating values for tick in legend; 20 ticks
    ticks = []
    for x in range(1,21):
        ticks.append(x/10*round_up_to_nearest_10(max(df["total_of_publication_number"])))

    tick_label = []
    for x in range(1,21):
       tick_label.append(x/10*round_up_to_nearest_10(max(df["total_of_publication_number"])))

    #not in use yet
    legend_label = []
    for x in range(1,21):
        legend_label.append(f"{round(x/20*max(df['total_of_publication_number']),0)}%")
        #legend_label

    fig = df.plot(ax = ax,markersize = 100,column = 'total_of_publication_number', figsize=(20,10),
                                                     cmap=cmap,
                                                     # create a normalized colorbar
                                                     #cbar = plt.cm.ScalarMappable(norm=norm),  
                                                     legend = True,
                                                    
                                                     legend_kwds={'label': 'Number of Patents',
                                                                    #'title_fontsize':20,
                                                                    'orientation': "horizontal",
                                                                    'pad': 0.04,
                                                                    'fraction': 0.09,
                                                                   # 'norm': norm,
                                                                    'ticks':ticks
                                                                    #"fmt":"{:%}"
                                                                    
                                                                    },
                                                     missing_kwds={
                                                                 "color":"grey",
                                                                 "edgecolor":"black",
                                                                 "hatch":"---",
                                                                 "label":"Missing Values"
                                                                },
                                                     vmin=0, vmax=round_up_to_nearest_10(max(df["total_of_publication_number"]))
                                                     )


    #include no countries in the title
    #creation of string
    #preparation for no country entries
    nocountries_string =f"{str(nocountries.iloc[0][0])}: {str(round(nocountries.iloc[0][1],2))}% \n "
    #nocountries_string
    for i in range(len(nocountries)):
        if i == 0:
            continue
        nocountries_string = nocountries_string + f"{str(nocountries.iloc[i][0])}: {str(round(nocountries.iloc[i][1],2))}% \n "

    #ax_cbar = fig.colorbar(cbar, ax=ax)
    #plt.legend(title = "lol",loc = "lower center")
    
    from datetime import datetime
    now = datetime.now()
    plt.title(f"{now.strftime('%d/%m/%Y %H:%M')}", loc='right', fontsize=15, color='grey', style='italic')
    plt.tick_params(left = False, bottom = False)

    #preparing the making of a condition for how many of the patents are actually in the data
    data_av_pubnum = list(df["publication_number"].dropna())
    data_notav_pubnum = list(data["publication_number"])
    count = int(0)
    for x in data_av_pubnum:
        if x in data_notav_pubnum:
            count +=1
    #ax.axes.set_xticklabels(tick_label)
    if situational is not None:

        plt.title(f"Worldwide distribution of {status} patents\n{assignee} | family members:{len(situational[situational['publication_number'].notnull()])}", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
        
    else:
        #check
        if count == len(data_av_pubnum):
            plt.title(f"Worldwide distribution of {status} patents\n{assignee} | total claims:{df['len_claims'].sum()}", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
        else:
            plt.title(f"Worldwide distribution of {status} patents\n{assignee}", fontsize = 30, loc = "left",y = 1.01,style = 'italic')
    #make condition for how many of the patents are actually in the data
    
        
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    plt.show()


def plot_worldmap_for_each_family(df,status):
    if "total_of_publication_number" in df.columns:
        df.drop("total_of_publication_number",axis = 1, inplace = True)
    families = data_all_total["inpadoc_family_members"].unique()
    for i in families:
        if i is NA:
            continue
        else:
            
            situational_data = df[df["inpadoc_family_members"]== i]
            #create column with aggregated count of patents country-wise
            total_column = situational_data[situational_data['country'].values != ""]
            total_column = total_column[["publication_number","country"]].groupby('country').agg('count')
            total_column.rename(columns = {"publication_number" : "total_of_publication_number"}, inplace = True)
            total_column.reset_index(inplace = True)
            total_column["total_of_publication_number"] = total_column.apply(lambda x: NA if x["total_of_publication_number"] == 0 else x["total_of_publication_number"],axis = 1)
            situational_data = situational_data.merge(total_column, left_on = 'country', right_on = 'country', how='left')
            #data_all_total
            situational_data= world_map.merge(situational_data, left_on = 'COUNTRY', right_on = 'country', how='outer')
            situational_data['total_of_publication_number'] =situational_data['total_of_publication_number'].fillna(0)

            #create variables for text showoff
            text = [f"countries and their total count:\n"]
            unique_countries = situational_data.country.dropna().unique()
            for k in unique_countries:
                countries = k
                values = str(int(situational_data[situational_data["country"]==str(k)][["total_of_publication_number"]].agg('count')))
                text.append(f"{countries}: {values} \n")
            print(f"length of the family: {len(situational_data[situational_data['publication_number'].notnull()])} \nthe family: {i}\n{''.join(text)}\n{plot_worldmap(situational_data,situational_data,status)}")
        
    
    



<font size=20>Plot for all patents

In [ ]:
#plot for all patents (granted and not granted)
plot_worldmap_for_each_family(data_all_total,"all")


<font size=20>Plot for only granted patents

In [ ]:
#plot for only granted patents
plot_worldmap_for_each_family(data_granted_total,"granted")


<font size=20>Plot for only NOT-granted patents

In [ ]:
#plot for only NOT-granted patents
plot_worldmap_for_each_family(data_not_granted_total,"not granted")
